In [27]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.word2vec import LineSentence

In [28]:
with open('text8') as f:
    data = f.read()

In [29]:
print(data[0:12])

 anarchism o


In [42]:
data_sentences = []
sentences = []
for i in data.split():
    sentences.append(i)

    if len(sentences) == 50:
        data_sentences.append(sentences)
        sentences = []

In [43]:
print(data_sentences[0:11])

[['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the'], ['organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing'], ['interpretations', 'of', 'what', 'this', 'means', 'anarchism', 'also', 'refers', 'to', 'related'

In [48]:
hiperparameters = {
    'vector_size': [50, 100, 200],
    'sg': [0, 1], # 1 = skip-gram, 0 = CBOW
    'window': [2, 5, 10],
    'epochs': [5, 10, 20],
}

In [49]:
from sklearn.model_selection import ParameterGrid


grid = list(ParameterGrid(hiperparameters))

In [77]:
# abrir arquivo com frases para teste
test_vectors = []
target_words = []
with open("questions-words.txt") as f:
        for line in f:
            if line.startswith(":"):
                continue
            
            line = line.strip().lower().split(' ')

            if len(line) != 4:
                 continue
            
            test_vectors.append(line[0:3])
            target_words.append(line[3])      

print(len(test_vectors))      

19544


In [94]:

best_config = {}
min_avg_error = 50000

worst_config = {}
max_avg_error = 0

results = []

for parameter_configuration in grid:
    total_error = 0.0
    count_test_words_in_vocab = 0

    model = Word2Vec(sentences=data_sentences, vector_size=parameter_configuration['vector_size'], window=parameter_configuration['window'], sg=parameter_configuration['sg'], epochs=parameter_configuration['epochs'])

    for idx in range(len(test_vectors)):

        # Extract the words in the analogy
        word_a, word_b, word_c = test_vectors[idx]
        target = target_words[idx]

        # Check if all words are in the model's vocabulary
        if all(word in model.wv for word in [word_a, word_b, word_c, target]):
            # Calculate vector: vec_a - vec_b + vec_c
            analogy_vector = model.wv[word_a] - model.wv[word_b] + model.wv[word_c]
            
            # Calculate cosine similarity with the expected word (word_d)
            similarity = cosine_similarity([analogy_vector], [model.wv[target]])[0][0]

            # Calculate the error as 1 - similarity (closer to 0 means better)
            error = 1 - similarity
            total_error += error
            count_test_words_in_vocab += 1
            """ print(f"Analogy: {word_a} - {word_b} + {word_c} = {target}")
            print(f"Similarity with {target}: {similarity:.4f}, Error: {error:.4f}")
            print() """

    # Step 4: Calculate average error
    average_error = total_error / count_test_words_in_vocab if count_test_words_in_vocab > 0 else 0
    """ print(f"\nAverage Analogy Error: {average_error}") """

    results.append((parameter_configuration, average_error))

    if average_error < min_avg_error:
        best_config = parameter_configuration
        min_avg_error = average_error
        model.save("best_model")
    
    if average_error > max_avg_error:
        worst_config = parameter_configuration
        max_avg_error = average_error


print(f"\nBest Configuration: {best_config}")
print(f"Min Average Error: {min_avg_error}")

print(f"\nWorst Configuration: {worst_config}")
print(f"Max Average Error: {max_avg_error}")




Best Configuration: {'epochs': 5, 'sg': 1, 'vector_size': 50, 'window': 10}
Min Average Error: 0.55538132695304

Worst Configuration: {'epochs': 5, 'sg': 0, 'vector_size': 200, 'window': 5}
Max Average Error: 0.8978691275613163


In [95]:
print(results)

[({'epochs': 5, 'sg': 0, 'vector_size': 50, 'window': 2}, 0.8637289382061297), ({'epochs': 5, 'sg': 0, 'vector_size': 50, 'window': 5}, 0.8844702592285507), ({'epochs': 5, 'sg': 0, 'vector_size': 50, 'window': 10}, 0.8667843762558468), ({'epochs': 5, 'sg': 0, 'vector_size': 100, 'window': 2}, 0.8777844809484876), ({'epochs': 5, 'sg': 0, 'vector_size': 100, 'window': 5}, 0.8917970729349668), ({'epochs': 5, 'sg': 0, 'vector_size': 100, 'window': 10}, 0.8814665731799214), ({'epochs': 5, 'sg': 0, 'vector_size': 200, 'window': 2}, 0.8836903947134177), ({'epochs': 5, 'sg': 0, 'vector_size': 200, 'window': 5}, 0.8978691275613163), ({'epochs': 5, 'sg': 0, 'vector_size': 200, 'window': 10}, 0.8830664120799637), ({'epochs': 5, 'sg': 1, 'vector_size': 50, 'window': 2}, 0.6263788621831115), ({'epochs': 5, 'sg': 1, 'vector_size': 50, 'window': 5}, 0.5774646656031192), ({'epochs': 5, 'sg': 1, 'vector_size': 50, 'window': 10}, 0.55538132695304), ({'epochs': 5, 'sg': 1, 'vector_size': 100, 'window': 2

In [98]:
# store results sorted by error
results.sort(key=lambda x: x[1])

with open("results.txt", "w") as f:
    for result in results:
        f.write(f"{result[0]} -> {result[1]}\n")
